In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

!pip install transformers torch torchvision advertorch pillow matplotlib tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for advertorch: filename=advertorch-0.2.3-py3-none-any.whl size=5696198 sha256=d8cadc1693873b5e5aa63bfb7f9da297223973c078011b89bd2f7914281bb360
  Stored in directory: /root/.cache/pip/wheels/1b/00/de/d11c024ea240dfcf62cd3a94e1653bbb26139777243701dbf5
Successfully built advertorch


In [ ]:
!pip install torchattacks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
  Attempting uninstall: requests
    Found existing installation: req

APGD COCO

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel
import matplotlib.pyplot as plt
from tqdm import tqdm
import torchattacks
import json
from google.colab import drive

drive.mount('/content/drive')

# Paths
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/coco"
COCO_IMAGE_PATH = "/content/drive/MyDrive/Colab Notebooks/coco"
PERTURBED_IMAGE_PATH = "/content/drive/MyDrive/Colab Notebooks/Perturbed Images/APGD_COCO"
RESULTS_PATH = os.path.join(BASE_PATH, "results")
CAPTIONS_FILE = "/content/drive/MyDrive/Colab Notebooks/captions.json"

# Create output directories
os.makedirs(PERTURBED_IMAGE_PATH, exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

# Load captions
with open(CAPTIONS_FILE, 'r') as f:
    captions_data = json.load(f)

# Image preprocessing and postprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

postprocess = transforms.Compose([
    transforms.Normalize(mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
                          std=[1 / 0.229, 1 / 0.224, 1 / 0.225]),
    transforms.ToPILImage(),
])

class CLIPVisionWrapper(torch.nn.Module):
    def __init__(self, vision_model):
        super().__init__()
        self.vision_model = vision_model

    def forward(self, pixel_values):
        return self.vision_model(pixel_values=pixel_values).pooler_output

def apply_apgd_attack(vision_model, image_tensor, epsilon=0.1, num_iterations=10):
    vision_model.eval()
    attack = torchattacks.PGD(vision_model, eps=epsilon, alpha=epsilon / 5, steps=num_iterations)
    image_tensor = image_tensor.unsqueeze(0)
    dummy_label = torch.tensor([0])
    perturbed_image = attack(image_tensor, dummy_label)
    return torch.clamp(perturbed_image.squeeze(0), 0, 1)

def visualize_perturbation(original, perturbed, original_caption, perturbed_caption, output_path):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(original)
    ax[0].set_title(f"Original: {original_caption}")
    ax[0].axis("off")

    ax[1].imshow(perturbed)
    ax[1].set_title(f"Perturbed: {perturbed_caption}")
    ax[1].axis("off")

    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

def evaluate_captions(model, processor, captions_data, perturbed_images):
    success_count = 0
    total = len(captions_data)

    if total == 0:
        print("No images found for evaluation.")
        return 0.0, 0.0

    for data, perturbed_img in zip(captions_data, perturbed_images):
        original_captions = data['text'] if isinstance(data['text'], list) else [data['text']]

        with torch.no_grad():
            inputs = processor(images=perturbed_img, text=original_captions, return_tensors="pt", padding=True)
            outputs = model(**inputs)
            logits = outputs.logits_per_image

        predicted_caption = original_captions[logits.argmax()]
        if predicted_caption not in original_captions:
            success_count += 1

    success_rate = (success_count / total) * 100
    return success_rate

def main():
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
    vision_wrapper = CLIPVisionWrapper(clip_model.vision_model)

    original_images = []
    perturbed_images = []

    for entry in tqdm(captions_data):
        img_file = entry['image']
        original_caption = entry['text'] if isinstance(entry['text'], list) else [entry['text']]

        img_path = os.path.join(COCO_IMAGE_PATH, img_file)
        image = Image.open(img_path).convert("RGB")

        image_tensor = preprocess(image)
        perturbed_tensor = apply_apgd_attack(vision_wrapper, image_tensor)
        perturbed_image = postprocess(perturbed_tensor)

        original_images.append(image)
        perturbed_images.append(perturbed_image)

        perturbed_img_path = os.path.join(PERTURBED_IMAGE_PATH, img_file)
        perturbed_image.save(perturbed_img_path)

        visualization_path = os.path.join(PERTURBED_IMAGE_PATH, f"vis_{img_file}.png")
        visualize_perturbation(image, perturbed_image, original_caption[0], "Model Prediction", visualization_path)

    success_rate = evaluate_captions(clip_model, processor, captions_data, perturbed_images)

    results = {
        "success_rate": success_rate
    }

    with open(os.path.join(RESULTS_PATH, "evaluation_results.json"), "w") as f:
        json.dump(results, f, indent=4)

    print("Evaluation completed.")
    print(f"Attack Success Rate: {success_rate:.2f}%")

if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


100%|██████████| 12/12 [08:21<00:00, 41.76s/it]


Evaluation completed.
Attack Success Rate: 0.00%


APGD VQA

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

# Paths
VQA_IMAGE_PATH = "/content/drive/MyDrive/Colab Notebooks/textvqa"
PERTURBED_IMAGE_PATH = "/content/drive/MyDrive/Colab Notebooks/Perturbed Images/APGD_VQA"
VQA_JSON_PATH = "/content/drive/MyDrive/Colab Notebooks/vqa.json"

# Create output directory
os.makedirs(PERTURBED_IMAGE_PATH, exist_ok=True)

# Image preprocessing and postprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

postprocess = transforms.Compose([
    transforms.Normalize(mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
                          std=[1 / 0.229, 1 / 0.224, 1 / 0.225]),
    transforms.ToPILImage(),
])

def apply_perturbation(image_tensor, epsilon=0.1):
    """Apply perturbation to the image tensor."""
    noise = torch.randn_like(image_tensor) * epsilon
    perturbed_image = image_tensor + noise
    perturbed_image = torch.clamp(perturbed_image, 0, 1)  # Ensure pixel values are valid
    return perturbed_image

def visualize_perturbation(original, perturbed, question, original_answer, output_path):
    """Visualize the original and perturbed images side by side."""
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(original)
    ax[0].set_title(f"Original: {question}\nAnswer: {original_answer}")
    ax[0].axis("off")

    ax[1].imshow(perturbed)
    ax[1].set_title("Perturbed Image")
    ax[1].axis("off")

    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

def main():
    # Load pre-trained CLIP model
    model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

    # Load VQA data
    with open(VQA_JSON_PATH, 'r') as f:
        vqa_data = json.load(f)['data']  # Access the 'data' key to get the list of entries

    # Process each entry in the VQA dataset
    for entry in tqdm(vqa_data):
        img_file = f"{entry['image_id']}.jpg"
        question = entry['question']
        original_answer = entry['answers'][0]  # Use the first answer as the original

        img_path = os.path.join(VQA_IMAGE_PATH, img_file)
        if not os.path.exists(img_path):
            print(f"Image {img_file} not found, skipping.")
            continue

        image = Image.open(img_path).convert("RGB")

        # Preprocess image
        image_tensor = preprocess(image)

        # Apply perturbation
        perturbed_tensor = apply_perturbation(image_tensor)
        perturbed_image = postprocess(perturbed_tensor)

        # Save perturbed image
        perturbed_img_path = os.path.join(PERTURBED_IMAGE_PATH, img_file)
        perturbed_image.save(perturbed_img_path)

        # Visualize original vs perturbed
        visualization_path = os.path.join(PERTURBED_IMAGE_PATH, f"vis_{img_file}.png")
        visualize_perturbation(image, perturbed_image, question, original_answer, visualization_path)

if __name__ == "__main__":
    main()




100%|██████████| 9/9 [00:04<00:00,  1.91it/s]


UNIVERSAL VQA

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.1.3 requires requests>=2.30.0, but you have requests 2.25.1 which is incompatible.


In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
import gradio as gr
import json

# Paths
VQA_IMAGE_PATH = "/content/drive/MyDrive/Colab Notebooks/textvqa"
PERTURBED_IMAGE_PATH = "/content/drive/MyDrive/Colab Notebooks/Perturbed Images/Universal_VQA"

# Create output directory
os.makedirs(PERTURBED_IMAGE_PATH, exist_ok=True)

# Preprocessing and Postprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

postprocess = transforms.Compose([
    transforms.Normalize(mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
                          std=[1 / 0.229, 1 / 0.224, 1 / 0.225]),
    transforms.ToPILImage(),
])

def apply_perturbation(image_tensor, epsilon=0.1):
    """
    Apply perturbation to the image tensor.
    """
    noise = torch.randn_like(image_tensor) * epsilon
    perturbed_image = image_tensor + noise
    perturbed_image = torch.clamp(perturbed_image, 0, 1)  # Ensure valid pixel values
    return perturbed_image

def generate_perturbations(vqa_data):
    """
    Generate perturbed images and save them for use in Gradio examples.
    """
    examples = []
    for entry in vqa_data:
        img_file = f"{entry['image_id']}.jpg"
        img_path = os.path.join(VQA_IMAGE_PATH, img_file)

        if not os.path.exists(img_path):
            print(f"Image {img_file} not found. Skipping...")
            continue

        # Open the original image
        image = Image.open(img_path).convert("RGB")
        image_tensor = preprocess(image)

        # Apply perturbation
        perturbed_tensor = apply_perturbation(image_tensor)
        perturbed_image = postprocess(perturbed_tensor)

        # Save perturbed image
        perturbed_img_path = os.path.join(PERTURBED_IMAGE_PATH, img_file)
        perturbed_image.save(perturbed_img_path)

        # Add example to Gradio
        examples.append([perturbed_img_path, entry["question"]])

    return examples

def test_question(perturbed_image_path, question):
    """
    Function to test perturbed images with a given question.
    """
    perturbed_image = Image.open(perturbed_image_path).convert("RGB")

    # Simulating output (replace this with inference logic if necessary)
    simulated_answer = "This is a perturbed output for the question."
    return perturbed_image, f"Question: {question}\nAnswer: {simulated_answer}"

def main():
    # Load VQA data
    vqa_data_path = "/content/drive/MyDrive/Colab Notebooks/vqa.json"
    with open(vqa_data_path, "r") as f:
        vqa_data = json.load(f)["data"]

    # Generate perturbations
    examples = generate_perturbations(vqa_data)

    # Gradio interface
    interface = gr.Interface(
        fn=test_question,
        inputs=[
            gr.Image(label="Perturbed Image", type="filepath"),
            gr.Textbox(label="Question")
        ],
        outputs=[
            gr.Image(label="Perturbed Image"),
            gr.Textbox(label="Result")
        ],
        examples=examples,
        title="Universal Perturbations for TextVQA",
        description="Upload a perturbed image, ask a question, and analyze the results."
    )

    # Launch interface
    interface.launch()

if __name__ == "__main__":
    main()



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e2d860998b95633282.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel
from torchattacks import CW
from tqdm import tqdm
import matplotlib.pyplot as plt
import random

# Paths
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/llava_targeted"
IMAGE_PATH = "/content/drive/MyDrive/Colab Notebooks/coco"
CW_OUTPUT_PATH = os.path.join(BASE_PATH, "CW_Targeted")
SEMANTIC_OUTPUT_PATH = os.path.join(BASE_PATH, "Semantic_Targeted")

# Create directories for saving adversarial images
os.makedirs(CW_OUTPUT_PATH, exist_ok=True)
os.makedirs(SEMANTIC_OUTPUT_PATH, exist_ok=True)

# Preprocessing for the model
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

postprocess = transforms.Compose([
    transforms.Normalize(mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
                          std=[1 / 0.229, 1 / 0.224, 1 / 0.225]),
    transforms.ToPILImage(),
])

# Targeted CW Attack
def apply_cw_attack(model, image_tensor, target_class, c=1e-4, kappa=0, steps=1000):
    """
    Apply CW targeted attack.
    """
    model.eval()
    attack = CW(model, c=c, kappa=kappa, steps=steps)
    image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension
    target_labels = torch.tensor([target_class])  # Targeted class label
    perturbed_image = attack(image_tensor, target_labels)
    return torch.clamp(perturbed_image.squeeze(0), 0, 1)  # Clamp values

# Semantic Attack
def apply_semantic_attack(image_tensor, intensity=0.1):
    """
    Apply a semantic attack by adding color jitter or slight transformations.
    """
    noise = torch.randn_like(image_tensor) * intensity
    perturbed_image = image_tensor + noise
    return torch.clamp(perturbed_image, 0, 1)

# Visualize and Save Images
def save_image(original, perturbed, path, attack_name, img_name):
    """
    Save original and perturbed images side by side.
    """
    perturbed_image = postprocess(perturbed)
    perturbed_image.save(os.path.join(path, f"{attack_name}_{img_name}"))

# Main Function
def main():
    # Load the pre-trained LLaVA model (replace CLIP with LLaVA once integrated)
    model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

    # Targeted misclassification labels (choose random target classes)
    target_classes = [random.randint(0, 999) for _ in range(10)]  # Example: ImageNet classes

    # Process each image in the dataset
    for img_file in tqdm(os.listdir(IMAGE_PATH)):
        if not img_file.endswith(('.png', '.jpg', '.jpeg')):
            continue

        # Load image
        img_path = os.path.join(IMAGE_PATH, img_file)
        image = Image.open(img_path).convert("RGB")

        # Preprocess image
        image_tensor = preprocess(image)

        # Targeted CW Attack
        target_class = random.choice(target_classes)  # Choose a random target class
        perturbed_cw = apply_cw_attack(model, image_tensor, target_class)
        save_image(image, perturbed_cw, CW_OUTPUT_PATH, "CW", img_file)

        # Semantic Attack
        perturbed_semantic = apply_semantic_attack(image_tensor, intensity=0.1)
        save_image(image, perturbed_semantic, SEMANTIC_OUTPUT_PATH, "Semantic", img_file)

    print(f"Targeted CW and Semantic attacks completed. Adversarial images saved in:\n- {CW_OUTPUT_PATH}\n- {SEMANTIC_OUTPUT_PATH}")

if __name__ == "__main__":
    main()


  0%|          | 0/13 [00:00<?, ?it/s]


ValueError: You have to specify pixel_values

In [ ]:
import os
import random
import torch
from PIL import Image
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

# Paths
BASE_PATH = "/content/drive/MyDrive/Colab Notebooks/coco"
IMAGE_PATH =  "/content/drive/MyDrive/Colab Notebooks/coco"
CW_OUTPUT_PATH = os.path.join(BASE_PATH, "perturbed/CW")
RESULTS_PATH = os.path.join(BASE_PATH, "results")
CAPTIONS_FILE = "/content/drive/MyDrive/Colab Notebooks/captions.json"

# Create output directories
os.makedirs(CW_OUTPUT_PATH, exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

# Load captions
with open(CAPTIONS_FILE, "r") as f:
    captions_data = json.load(f)

# Image preprocessing
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

postprocess = transforms.Compose([
    transforms.Normalize(mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
                          std=[1 / 0.229, 1 / 0.224, 1 / 0.225]),
    transforms.ToPILImage(),
])

# CW Attack Implementation
def apply_cw_attack(model, image_tensor, target_class, c=1e-4, kappa=0, steps=1000):
    """Apply Carlini-Wagner L2 targeted attack."""
    from torchattacks import CW
    model.eval()
    attack = CW(model, c=c, kappa=kappa, steps=steps)
    image_tensor = image_tensor.unsqueeze(0)  # Add batch dimension
    target_labels = torch.tensor([target_class])  # Targeted class label
    perturbed_image = attack(image_tensor, target_labels)
    return torch.clamp(perturbed_image.squeeze(0), 0, 1)  # Clamp values

# Save perturbed image and visualization
def save_image(original, perturbed, output_path, image_name, original_caption, malicious_caption):
    """Save perturbed image and its visualization."""
    # Save perturbed image
    perturbed_img_path = os.path.join(output_path, image_name)
    perturbed.save(perturbed_img_path)

    # Create visualization
    vis_path = os.path.join(output_path, f"vis_{image_name}")
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(original)
    ax[0].set_title(f"Original: {original_caption}")
    ax[0].axis("off")

    ax[1].imshow(perturbed)
    ax[1].set_title(f"Perturbed: {malicious_caption}")
    ax[1].axis("off")

    plt.tight_layout()
    plt.savefig(vis_path)
    plt.close()

# Generate malicious captions
def generate_malicious_caption():
    """Generate a random malicious caption."""
    prompts = [
        "Click here to claim your prize!",
        "Visit example.com for a free giveaway!",
        "Congratulations, you've won!",
        "Your account has been compromised! Go to phishing-site.com.",
        "Exclusive offer available at malicious-link.com!"
    ]
    return random.choice(prompts)

# Main function
def main():
    # Load pre-trained CLIP model
    model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

    # Target classes for attack
    target_classes = list(range(1000))  # Randomly choose valid target labels

    for entry in tqdm(captions_data):
        img_file = entry["image"]
        original_caption = entry["text"][0] if isinstance(entry["text"], list) else entry["text"]
        malicious_caption = generate_malicious_caption()

        img_path = os.path.join(IMAGE_PATH, img_file)
        if not os.path.exists(img_path):
            print(f"Image {img_file} not found, skipping.")
            continue

        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_file}: {e}")
            continue

        image_tensor = preprocess(image)

        # Targeted CW Attack
        target_class = random.choice(target_classes)  # Choose a random target class
        perturbed_cw = apply_cw_attack(model, image_tensor, target_class)

        # Postprocess perturbed image
        perturbed_image = postprocess(perturbed_cw)

        # Save perturbed image and visualizations
        save_image(image, perturbed_image, CW_OUTPUT_PATH, img_file, original_caption, malicious_caption)

if __name__ == "__main__":
    main()


# Main function
def main():
    # Load pre-trained CLIP model
    model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

    # Target classes for attack
    target_classes = list(range(1000))  # Randomly choose valid target labels

    for entry in tqdm(captions_data):
        img_file = entry["image"]
        original_caption = entry["text"][0] if isinstance(entry["text"], list) else entry["text"]
        malicious_caption = generate_malicious_prompt()  # Generate unique malicious prompt

        img_path = os.path.join(IMAGE_PATH, img_file)
        if not os.path.exists(img_path):
            print(f"Image {img_file} not found, skipping.")
            continue

        image = Image.open(img_path).convert("RGB")
        image_tensor = preprocess(image)

        # Targeted CW Attack
        target_class = random.choice(target_classes)  # Choose a random target class
        perturbed_cw = apply_cw_attack(model, image_tensor, target_class)

        # Postprocess perturbed image
        perturbed_image = postprocess(perturbed_cw)

        # Save perturbed image and visualizations
        save_image(image, perturbed_image, CW_OUTPUT_PATH, "CW", img_file, original_caption, malicious_caption)

if __name__ == "__main__":
    main()


  0%|          | 0/12 [00:00<?, ?it/s]


ValueError: You have to specify pixel_values